### Original example from: https://github.com/cajal/microns_phase3_nda/blob/v8/tutorial_notebooks/Matched_Cell_Functional_Data.ipynb

In [1]:
import os
# running from Root directory
os.chdir("..")

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils, skeletonize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from caveclient import CAVEclient
import meshparty.skeleton_io

In [3]:
client = CAVEclient('minnie65_public')

In [6]:
# client.materialize.get_tables()
co_df = client.materialize.query_table('coregistration_auto_phase3_fwd_apl_vess_combined')
co_df.head()
# co_df.to_csv('coregistration_auto_phase3_fwd_apl_vess_combined.csv')

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,388873,2020-09-28 22:45:17.303349+00:00,t,357.495603,97818258123783846,864691135938031925,1,2024-05-23 16:26:39.793351+00:00,t,388873,4,7,107,1,18.1918,1.88120,"[240496, 103184, 15068]","[nan, nan, nan]","[nan, nan, nan]"
1,357184,2020-09-28 22:45:21.668338+00:00,t,398.268826,95566801907418600,864691135730121401,2,2024-05-23 16:26:39.793945+00:00,t,357184,4,7,127,1,13.0387,-7.90215,"[223792, 106080, 15013]","[nan, nan, nan]","[nan, nan, nan]"
2,357165,2020-09-28 22:44:28.479993+00:00,t,254.621450,94792402124187373,864691135737064068,3,2024-05-23 16:26:39.794546+00:00,t,357165,4,7,152,1,15.7229,3.58140,"[218304, 103136, 15112]","[nan, nan, nan]","[nan, nan, nan]"
3,290596,2020-09-28 22:41:22.519134+00:00,t,251.790459,89656239713364965,864691135338589414,4,2024-05-23 16:26:39.795104+00:00,t,290596,4,7,243,1,24.0639,3.89930,"[181136, 108880, 15012]","[nan, nan, nan]","[nan, nan, nan]"
4,578537,2020-09-28 22:41:28.675248+00:00,t,246.512845,112031301338636911,864691135291722934,5,2024-05-23 16:26:39.795684+00:00,t,578537,4,7,381,1,18.6186,5.59030,"[343616, 92608, 15054]","[nan, nan, nan]","[nan, nan, nan]"


In [22]:
neurons_df = pd.read_csv('data/connectome_neurons.csv')
neurons_df.head()

,Unnamed: 0,volume,clf_type,cell_type,mtype,pre_synapses,post_synapses,ds_num_of_post_synapses,ds_num_of_pre_synapses,ds_pre_syn_weight,root_id,pre_syn_weight,ex_pre_syn_weight,inh_pre_syn_weight
0,0,234.475029,E,6P-IT,L6tall-c,1,0,3,50,4245.760000,864691134784325628,1096.000000,1096.000000,NaN
1,1,313.159680,E,6P-IT,L6tall-c,0,1,4,68,4804.588235,864691134807855033,NaN,NaN,NaN
2,2,318.771200,E,6P-IT,L6short-a,50,5,9,1661,6338.362432,864691134884743162,5991.680000,6356.888889,5562.956522
3,3,207.237120,E,6P-IT,L6short-b,25,17,28,659,7949.116844,864691134884747514,4935.680000,5514.800000,2619.200000
4,4,205.180109,E,6P-CT,L6tall-a,95,330,557,1779,6489.351321,864691134884754426,5751.284211,5548.327869,6115.411765


In [28]:
root_ids = set(co_df.pt_root_id)
print(len(root_ids))

n_root_ids = set(neurons_df.root_id)
print(len(n_root_ids))

ids = root_ids.intersection(n_root_ids) 
print(len(ids))



36505
71736
34726


### Sort post-synaptic neurons 
sort by how many pre-synaptic neurons with functional / acitivty data 

### Get functional data

In [42]:
# from microns_phase3 import nda, utils

In [40]:
sample_neuron = list(ids)[0]
# entry = co_df[co_df.pt_root_id == sample_neuron].iloc[0]
# unit_key = entry[['session', 'scan_idx', 'unit_id']]
# dict(unit_key)